### Collection

In [1]:
import os
from src.StreamPort.device.analyses import MassSpecAnalyses

path = "C:/Users/Sandeep/Desktop/Error-LC/Method-Data"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

error_lc_files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    error_lc_files.extend(batch_files)

analyses = MassSpecAnalyses(files=error_lc_files)
print("Number of analyses: ", len(analyses.data))

No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43\016-NV-Standby.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12\016-NV-Standby.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17\064-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250617_Mix-1_50x10ng-mL 2025-06-17 09-19-10\064-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34\034-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58\034-NV-StandBy.D
No MS2 data found in directory: C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42\028-NV-StandBy.D
No MS2 data found in d

mzs available in SIM: 120, 124, 134, 237, 247, 254, 258, 267, 268, 275, 278, 296, 300, 325, 370, 429, 441, 748, 752

mzs available in TIC: range(100, 801)

In [2]:
chromas = analyses.plot_data(indices=7, data="sim", dim=3, mz=237, rt=None) # defaults: indices=None(all), data="SIM", dim=None(2), mz=None(mz[0]), rt=None(rt[:])
chromas.show()

### Feature Extraction

In [3]:
from src.StreamPort.device.methods import MassSpecDataMethodExtractFeaturesNative
import plotly.graph_objects as go

processor = MassSpecDataMethodExtractFeaturesNative(rt=3.223, mz=238)
gen = processor.run(analyses)

[WARNING] Gaussian fit failed for tile (rt[3.064416666666667:3.3776333333333333], mz[134.0:237.0]: Optimal parameters not found: Number of calls to function has reached maxfev = 1200.. Returning the debug_figure.


ValueError: operands could not be broadcast together with shapes (466,) (22,) 

In [ ]:
features = analyses.get_features()
features.describe()

In [ ]:
features_plot = analyses.plot_features()
features_plot.show()

### Diagnostics 

In [ ]:
import pandas as pd 
ana = analyses.data[0]
print("Name: ", ana["name"], "\nSIM-data: ", pd.DataFrame({"mzs" : ana["sim"]["mz"],"compounds" : ana["sim"]["compound"]}))


In [ ]:
data = ana["sim"]
print(data["rt"])

In [ ]:

rt = 0.05166667
mz = 237
if mz is not None:
    mz_index = data["mz"].tolist().index(mz) # find the index of the given mz
else:
    mz_index = 0
    mz = data["mz"][0]

if rt is not None:
    rt_array = data["rt"]
    # find closest value to given rt and plot it
    rt_index = abs(rt_array - rt).argmin() # returns index of closest rt to rt argument 
    y = (data["intensity"][:, mz_index]) * 0
    y[rt_index] = rt_array[rt_index]  
    # x = array([rt_array[rt_index]]                   
else:
    y = data["intensity"][:, mz_index] 
print("rt: ", rt_array, " ", rt_array.shape)
print("y: ", y, " ", y.shape)

### Method checks

In [ ]:

file = error_lc_files[2]
print(file)


import rainbow as rb
meta = rb.read_metadata(file)
print(meta)

datadir = rb.read(file)
print(datadir.metadata)


datafiles = datadir.datafiles
for f in datafiles: # xlabels= retention time, ylabels = mz/wavelength, data = intensity
    if f.name in ["MSD1.MS", "MSD2.MS"]:
        print(f.metadata)
        #print(f.get_info())

#print(f.extract_traces()) # returns data (intensity) for the given ylabels. Rows correspond to ylabels

f.plot(label=325) # label represents the ylabel (mz) for which data should be plotted